In [3]:
# !pip install opencv-python

In [4]:
# !pip install FuzzyTM 

In [5]:
# !pip install numpy 

In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv('fer2013.csv')
X_test = []
y_test = []
X_train = []
y_train = []
for index, row in df.iterrows():
  k = row['pixels'].split(" ")
  if row['Usage'] == "Training":
    X_train.append(np.array(k))
    y_train.append(row['emotion'])
  elif row['Usage'] == "PublicTest":
    X_test.append(np.array(k))
    y_test.append(row['emotion'])
X_train = np.array(X_train, dtype = 'uint8')
y_train = np.array(y_train, dtype = 'uint8')
X_test = np.array(X_test, dtype = 'uint8')
y_test = np.array(y_test, dtype = 'uint8')


In [7]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [8]:
X_train[0]

array([ 70,  80,  82, ..., 106, 109,  82], dtype=uint8)

In [9]:
from tensorflow import keras
from keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)
print("HELLO")

HELLO


In [10]:
X_train

array([[ 70,  80,  82, ..., 106, 109,  82],
       [151, 150, 147, ..., 193, 183, 184],
       [231, 212, 156, ...,  88, 110, 152],
       ...,
       [ 74,  81,  87, ..., 188, 187, 187],
       [222, 227, 203, ..., 136, 136, 134],
       [195, 199, 205, ...,   6,  15,  38]], dtype=uint8)

In [11]:
#Reshape Data

In [12]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)
print("HELLO")

HELLO


In [13]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode = 'nearest')
testgen = ImageDataGenerator(rescale=1./255)
datagen.fit(X_train)
batch_size = 64
print("HELLO")

HELLO


In [14]:
train_flow = datagen.flow(X_train, y_train, batch_size=batch_size)
test_flow = testgen.flow(X_test, y_test, batch_size=batch_size)
print("HELLO")

HELLO


In [15]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
print("HELLO")

HELLO


In [16]:
def FER_Model(input_shape=(48,48,1)):
    # first input model
    visible = Input(shape=input_shape, name='input')
    num_classes = 7
    #the 1-st block
    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)#the 2-nd block
    conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
    conv2_2 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)#the 3-rd block
    conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
    conv3_3 = BatchNormalization()(conv3_3)
    conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
    conv3_4 = BatchNormalization()(conv3_4)
    pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
    drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)#the 4-th block
    conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
    conv4_4 = BatchNormalization()(conv4_4)
    pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
    drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)

    #the 5-th block
    conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
    conv5_3 = BatchNormalization()(conv5_3)
    conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
    conv5_3 = BatchNormalization()(conv5_3)
    pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
    drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)#Flatten and output
    flatten = Flatten(name = 'flatten')(drop5_1)
    ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)# create model
    model = Model(inputs =visible, outputs = ouput)
    # summary layers
    print(model.summary())

    return model
print("HELLO")

HELLO


In [17]:
model = FER_Model()
opt = Adam(learning_rate=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print("HELLO")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 48, 48, 1)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 conv1_2 (Conv2D)            (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 64)       256       
 hNormalization)                                                 
                                                                 
 pool1_1 (MaxPooling2D)      (None, 24, 24, 64)        0     

In [18]:
num_epochs = 100
history = model.fit_generator(train_flow,
                              steps_per_epoch=len(X_train) / batch_size,
                              epochs=num_epochs,
                              verbose=1,
                              validation_data=test_flow,
                              validation_steps = len(X_test)/batch_size)

Epoch 1/100


C:\Users\ASUS\AppData\Local\Temp\ipykernel_31352\1040513408.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_flow,


448/448 [==============================] - 65s 120ms/step - loss: 2.0232 - accuracy: 0.2141 - val_loss: 1.9001 - val_accuracy: 0.2508
Epoch 2/100
448/448 [==============================] - 54s 120ms/step - loss: 1.7916 - accuracy: 0.2487 - val_loss: 1.7643 - val_accuracy: 0.2728
Epoch 3/100
448/448 [==============================] - 54s 120ms/step - loss: 1.7626 - accuracy: 0.2710 - val_loss: 1.7413 - val_accuracy: 0.2839
Epoch 4/100
448/448 [==============================] - 54s 120ms/step - loss: 1.7179 - accuracy: 0.3056 - val_loss: 1.6914 - val_accuracy: 0.3369
Epoch 5/100
448/448 [==============================] - 54s 119ms/step - loss: 1.6563 - accuracy: 0.3436 - val_loss: 1.6095 - val_accuracy: 0.3901
Epoch 6/100
448/448 [==============================] - 54s 119ms/step - loss: 1.5817 - accuracy: 0.3795 - val_loss: 1.6407 - val_accuracy: 0.4099
Epoch 7/100
448/448 [==============================] - 53s 119ms/step - loss: 1.5119 - accuracy: 0.4104 - val_loss: 1.4660 - val_accurac

In [19]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
from tensorflow.keras.models import model_from_json
model = model_from_json(open("model.json", "r").read())
model.load_weights('model.h5')

In [7]:
import cv2
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
import cv2

cap = cv2.VideoCapture(0)

while cap.isOpened():
    res, frame = cap.read()
    print(res)  # Print the value of res

    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [16]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.preprocessing.image import img_to_array

model = model_from_json(open("model.json", "r").read())
model.load_weights('model.h5')
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
if face_haar_cascade.empty():
    print("Error loading cascade classifier XML file.")

cap = cv2.VideoCapture(0)

while cap.isOpened():
    res, frame = cap.read()
    print(res)  # Print the value of res

    height, width, _ = frame.shape
    sub_img = frame[0:int(height/6), 0:int(width)]

    black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0
    res = cv2.addWeighted(sub_img, 0.77, black_rect, 0.23, 0)
    FONT = cv2.FONT_HERSHEY_SIMPLEX
    FONT_SCALE = 0.8
    FONT_THICKNESS = 2
    label_color = (10, 10, 255)
    label = "Detecting Emotions now..."
    label_dimension = cv2.getTextSize(label, FONT, FONT_SCALE, FONT_THICKNESS)[0]
    textX = int((res.shape[1] - label_dimension[0]) / 2)
    textY = int((res.shape[0] + label_dimension[1]) / 2)
    cv2.putText(res, label, (textX, textY), FONT, FONT_SCALE, (0, 0, 0), FONT_THICKNESS)

    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray_image)
    if frame is None:
        print("Frame is None. Check your camera connection or configuration.")
        break  # Break out of the loop if frame is None

    try:
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(255, 0, 0), thickness=2) #rect blue
            roi_gray = gray_image[y-5:y+h+5, x-5:x+w+5] #region of interest
            roi_gray = cv2.resize(roi_gray, (48, 48)) #resize
            image_pixels = img_to_array(roi_gray) #arrayfication
            image_pixels = np.expand_dims(image_pixels, axis=0) #dim padding
            image_pixels /= 255 #norm
            predictions = model.predict(image_pixels)
            max_index = np.argmax(predictions)
            # print(predictions[1]) out of bounds
            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]
            cv2.putText(res, "Sentiment: {}".format(emotion_prediction), (0, textY + 22 + 5), FONT, 0.7, label_color, 2)
            label_violation = 'Confidence: {}'.format(str(np.round(np.max(predictions[0]) * 100, 1)) + "%")
            violation_text_dimension = cv2.getTextSize(label_violation, FONT, FONT_SCALE, FONT_THICKNESS)[0]
            violation_x_axis = int(res.shape[1] - violation_text_dimension[0])
            cv2.putText(res, label_violation, (violation_x_axis, textY + 22 + 5), FONT, 0.7, label_color, 2)
    except Exception as e:
        print("An exception occurred:", e)

    frame[0:int(height/6), 0:int(width)] = res
    cv2.imshow('frame', frame)

    key = cv2.waitKey(1) & 0xFF   
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


True
1/1 [==============================] - 0s 270ms/step
True
1/1 [==============================] - 0s 25ms/step
True
1/1 [==============================] - 0s 24ms/step
True
1/1 [==============================] - 0s 23ms/step
True
1/1 [==============================] - 0s 23ms/step
True
1/1 [==============================] - 0s 24ms/step
True
1/1 [==============================] - 0s 23ms/step
True
1/1 [==============================] - 0s 21ms/step
True
1/1 [==============================] - 0s 22ms/step
True
1/1 [==============================] - 0s 23ms/step
True
1/1 [==============================] - 0s 24ms/step
True
1/1 [==============================] - 0s 22ms/step
True
1/1 [==============================] - 0s 24ms/step
True
1/1 [==============================] - 0s 22ms/step
True
1/1 [==============================] - 0s 22ms/step
True
1/1 [==============================] - 0s 22ms/step
True
1/1 [==============================] - 0s 23ms/step
True
1/1 [====================

In [3]:
import os
import time
import cv2
import numpy as np
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.preprocessing.image import img_to_array

# Load the model
model = model_from_json(open("model.json", "r").read())
model.load_weights('model.h5')

# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
if face_cascade.empty():
    print("Error loading cascade classifier XML file.")

# Start capturing video from the camera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    res, frame = cap.read()
    if not res:
        print("Failed to capture frame")
        break

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Iterate over each detected face
    for (x, y, w, h) in faces:
        # Extract the face ROI (Region of Interest)
        face_roi = gray_frame[y:y+h, x:x+w]
        
        # Resize the face ROI to match the input shape of the model
        resized_face = cv2.resize(face_roi, (48, 48))
        
        # Convert the resized face to an array
        image_pixels = img_to_array(resized_face)
        
        # Expand dimensions to match the model input shape
        image_pixels = np.expand_dims(image_pixels, axis=0)
        
        # Normalize the image
        image_pixels /= 255.0
        
        # Perform prediction
        predictions = model.predict(image_pixels)
        
        # Get the predicted class
        predicted_class = np.argmax(predictions[0])
        
        # Map predicted class to emotion label
        emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        emotion_label = emotion_labels[predicted_class]
        print(emotion_label,"| time: ", time.strftime("%H:%M:%S", time.localtime()))
        
        # Display the emotion label on the frame
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('frame', frame)
    
    # Check if the user pressed 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 449ms/step
Neutral time:  17:58:08
1/1 [==============================] - 0s 38ms/step
Neutral time:  17:58:08
1/1 [==============================] - 0s 44ms/step
Neutral time:  17:58:08
1/1 [==============================] - 0s 36ms/step
Neutral time:  17:58:08
1/1 [==============================] - 0s 37ms/step
Neutral time:  17:58:08
1/1 [==============================] - 0s 38ms/step
Neutral time:  17:58:08
1/1 [==============================] - 0s 47ms/step
Neutral time:  17:58:09
1/1 [==============================] - 0s 40ms/step
Neutral time:  17:58:09
1/1 [==============================] - 0s 38ms/step
Neutral time:  17:58:09
1/1 [==============================] - 0s 38ms/step
Neutral time:  17:58:09
1/1 [==============================] - 0s 37ms/step
Neutral time:  17:58:09
1/1 [==============================] - 0s 38ms/step
Neutral time:  17:58:09
1/1 [==============================] - 0s 40ms/step
Neutral time:  17:58:09
1/1 [======

In [1]:
cap.release()
cv2.destroyAllWindows

NameError: name 'cap' is not defined